In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline
import seaborn as sn

In [2]:
import geopandas as gpd
import pandas as pd

## Load daily temperature 

In [3]:
years = [year for year in range(1961, 2022)]

In [4]:
pattern = []
for year in years:
    temp_year = ddf.read_csv(r"/global/cfs/cdirs/m1532/Projects_MVP/geospatial/climate_heatwave/city_heatwave/city_climate/city_climate_" + str(year) + ".csv", dtype={'year': int}).compute().drop(columns={'Unnamed: 0'})
    pattern.append(temp_year)

In [5]:
temp = pd.concat(pattern)
temp.head()

,city_name,longitude,latitude,station_coordinate,year,month,date,elevation,mean_temp,min_temp,...,station_pressure,visibility,wind_speed,precipitation,fog,rain,snow,hail,thunder,tornado
0,Seattle,-122.323549,47.622281,"['99999924244', 47.683, -122.267]",1961,Jan,1961-01-01,14.9,41.2,37.9,...,23.6,9.9,1.7,NaN,0,1,0,0,0,0
1,Seattle,-122.323549,47.622281,"['99999924244', 47.683, -122.267]",1961,Jan,1961-01-02,14.9,36.4,30.9,...,26.9,9.2,1.9,0.0,1,0,0,0,0,0
2,Seattle,-122.323549,47.622281,"['99999924244', 47.683, -122.267]",1961,Jan,1961-01-03,14.9,35.8,30.9,...,23.0,9.6,4.2,0.0,1,0,0,0,0,0
3,Seattle,-122.323549,47.622281,"['99999924244', 47.683, -122.267]",1961,Jan,1961-01-04,14.9,41.8,39.9,...,20.0,11.0,7.8,NaN,0,1,0,0,0,0
4,Seattle,-122.323549,47.622281,"['99999924244', 47.683, -122.267]",1961,Jan,1961-01-05,14.9,44.3,41.0,...,13.2,4.5,3.6,NaN,0,1,0,0,0,0


In [6]:
temp['year'].unique()

array([1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021])

In [7]:
import math

def water_vapor_pressure(temperature):
    e = 0.61094 * math.exp(17.625 * temperature / (temperature + 243.04))
    return e


def apparent_temperature(temperature, water_vapor_pressure):
    A = -1.3 + 0.92 * temperature + 2.2 * water_vapor_pressure
    return A


temperature = 28 
water_vapor_pressure = water_vapor_pressure(temperature) 
apparent_temp = apparent_temperature(temperature, water_vapor_pressure)
print(f"Apparent temperature at {temperature} degrees Celsius and {water_vapor_pressure:.2f} kPa water vapor pressure is {apparent_temp:.2f} °C")

Apparent temperature at 28 degrees Celsius and 3.77 kPa water vapor pressure is 32.76 °C


In [8]:
def findApparentTemp(temperature):
    e = 0.61094 * math.exp(17.625 * temperature / (temperature + 243.04))
    A = -1.3 + 0.92 * temperature + 2.2 * e
    return A

In [9]:
climate = temp[['year', 'month', 'date', 'city_name', 'mean_temp', 'min_temp', 'max_temp']]
climate.head()

,year,month,date,city_name,mean_temp,min_temp,max_temp
0,1961,Jan,1961-01-01,Seattle,41.2,37.9,45.0
1,1961,Jan,1961-01-02,Seattle,36.4,30.9,44.1
2,1961,Jan,1961-01-03,Seattle,35.8,30.9,44.1
3,1961,Jan,1961-01-04,Seattle,41.8,39.9,45.0
4,1961,Jan,1961-01-05,Seattle,44.3,41.0,46.9


In [10]:
def fahrenheit_to_celsius(f):
    c = (f -32)*5/9
    return c

In [11]:
climate['mean_temp'] = climate['mean_temp'].apply(fahrenheit_to_celsius)
climate['min_temp'] = climate['min_temp'].apply(fahrenheit_to_celsius)
climate['max_temp'] = climate['max_temp'].apply(fahrenheit_to_celsius)
climate.head()

/tmp/ipykernel_371365/3470054733.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  climate['mean_temp'] = climate['mean_temp'].apply(fahrenheit_to_celsius)
/tmp/ipykernel_371365/3470054733.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  climate['min_temp'] = climate['min_temp'].apply(fahrenheit_to_celsius)
/tmp/ipykernel_371365/3470054733.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

,year,month,date,city_name,mean_temp,min_temp,max_temp
0,1961,Jan,1961-01-01,Seattle,5.111111,3.277778,7.222222
1,1961,Jan,1961-01-02,Seattle,2.444444,-0.611111,6.722222
2,1961,Jan,1961-01-03,Seattle,2.111111,-0.611111,6.722222
3,1961,Jan,1961-01-04,Seattle,5.444444,4.388889,7.222222
4,1961,Jan,1961-01-05,Seattle,6.833333,5.000000,8.277778


In [12]:
climate['AT_mean'] = climate['mean_temp'].apply(findApparentTemp)
climate['AT_min'] = climate['min_temp'].apply(findApparentTemp)
climate['AT_max'] = climate['max_temp'].apply(findApparentTemp)
climate.head()

/tmp/ipykernel_371365/3394420915.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  climate['AT_mean'] = climate['mean_temp'].apply(findApparentTemp)
/tmp/ipykernel_371365/3394420915.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  climate['AT_min'] = climate['min_temp'].apply(findApparentTemp)
/tmp/ipykernel_371365/3394420915.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

,year,month,date,city_name,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
0,1961,Jan,1961-01-01,Seattle,5.111111,3.277778,7.222222,5.334537,3.414893,7.579652
1,1961,Jan,1961-01-02,Seattle,2.444444,-0.611111,6.722222,2.550811,-0.576562,7.044359
2,1961,Jan,1961-01-03,Seattle,2.111111,-0.611111,6.722222,2.206584,-0.576562,7.044359
3,1961,Jan,1961-01-04,Seattle,5.444444,4.388889,7.222222,5.686472,4.575143,7.579652
4,1961,Jan,1961-01-05,Seattle,6.833333,5.000000,8.277778,7.163117,5.217431,8.717369


In [13]:
month_replace = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10,
                'Nov': 11, 'Dec': 12}

In [14]:
climate['month'] = climate['month'].replace(month_replace)
climate.head()

/tmp/ipykernel_371365/1066241071.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  climate['month'] = climate['month'].replace(month_replace)


,year,month,date,city_name,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
0,1961,1,1961-01-01,Seattle,5.111111,3.277778,7.222222,5.334537,3.414893,7.579652
1,1961,1,1961-01-02,Seattle,2.444444,-0.611111,6.722222,2.550811,-0.576562,7.044359
2,1961,1,1961-01-03,Seattle,2.111111,-0.611111,6.722222,2.206584,-0.576562,7.044359
3,1961,1,1961-01-04,Seattle,5.444444,4.388889,7.222222,5.686472,4.575143,7.579652
4,1961,1,1961-01-05,Seattle,6.833333,5.000000,8.277778,7.163117,5.217431,8.717369


### After data imputation, 97% of the counties have daily apparent tempeature data from 1960 to 2020

In [15]:
city_numbers = []
for year in years:
    dict_year = {}
    df_year = climate[climate['year'] == year]
    df_new = df_year.dropna(subset = 'AT_mean')
    numbers = len(df_new['city_name'].unique())
    dict_year[year] = numbers
    city_numbers.append(dict_year)

In [16]:
city_numbers

[{1961: 50},
 {1962: 50},
 {1963: 50},
 {1964: 50},
 {1965: 50},
 {1966: 50},
 {1967: 50},
 {1968: 50},
 {1969: 50},
 {1970: 50},
 {1971: 50},
 {1972: 50},
 {1973: 50},
 {1974: 50},
 {1975: 50},
 {1976: 50},
 {1977: 50},
 {1978: 50},
 {1979: 50},
 {1980: 50},
 {1981: 50},
 {1982: 50},
 {1983: 50},
 {1984: 50},
 {1985: 50},
 {1986: 50},
 {1987: 50},
 {1988: 50},
 {1989: 50},
 {1990: 50},
 {1991: 50},
 {1992: 50},
 {1993: 50},
 {1994: 50},
 {1995: 50},
 {1996: 50},
 {1997: 50},
 {1998: 50},
 {1999: 50},
 {2000: 50},
 {2001: 50},
 {2002: 50},
 {2003: 50},
 {2004: 50},
 {2005: 50},
 {2006: 50},
 {2007: 50},
 {2008: 50},
 {2009: 50},
 {2010: 50},
 {2011: 50},
 {2012: 50},
 {2013: 50},
 {2014: 50},
 {2015: 50},
 {2016: 50},
 {2017: 50},
 {2018: 50},
 {2019: 50},
 {2020: 50},
 {2021: 50}]

In [17]:
climate['AT_mean'].describe()

count    1.243715e+06
mean     1.678788e+01
std      1.134313e+01
min     -3.075036e+01
25%      8.597108e+00
50%      1.741217e+01
75%      2.581311e+01
max      5.770207e+01
Name: AT_mean, dtype: float64

In [18]:
climate['year'].unique()

array([1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021])

In [19]:
climate = climate.sort_values(['city_name', 'year', 'month'])
climate

,year,month,date,city_name,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
17020,1961,1,1961-01-01,Albany,-3.277778,-7.000000,-0.388889,-3.259279,-6.943068,-0.351144
17021,1961,1,1961-01-02,Albany,-4.833333,-13.111111,-2.000000,-4.806708,-12.870244,-1.978794
17022,1961,1,1961-01-03,Albany,-7.777778,-12.611111,-2.611111,-7.705031,-12.389948,-2.592301
17023,1961,1,1961-01-04,Albany,-6.444444,-9.222222,-2.611111,-6.397263,-9.113764,-2.592301
17024,1961,1,1961-01-05,Albany,-7.500000,-15.888889,-2.611111,-7.433185,-15.526032,-2.592301
...,...,...,...,...,...,...,...,...,...,...
5449,2021,12,2021-12-27,Tulsa,15.777778,6.722222,22.777778,17.151448,7.044359,25.741472
5450,2021,12,2021-12-28,Tulsa,14.444444,10.000000,22.222222,15.601510,10.597245,25.028388
5451,2021,12,2021-12-29,Tulsa,7.666667,2.222222,22.222222,8.057404,2.321239,25.028388
5452,2021,12,2021-12-30,Tulsa,6.388889,2.222222,14.388889,6.688750,2.321239,15.537452


### find temperature data during 1981 to 2010

In [20]:
year_temp = [year for year in range(1981, 2011)]

In [21]:
temp = climate[climate['year'].isin(year_temp)]
temp

,year,month,date,city_name,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
16420,1981,1,1981-01-01,Albany,-13.833333,-17.277778,-8.222222,-13.562737,-16.846720,-8.139343
16421,1981,1,1981-01-02,Albany,-6.277778,-17.222222,-2.777778,-6.233263,-16.793977,-2.759267
16422,1981,1,1981-01-03,Albany,-16.222222,-19.500000,-11.611111,-15.843406,-18.951129,-11.427102
16423,1981,1,1981-01-04,Albany,-20.722222,-25.000000,-13.888889,-20.104458,-24.121853,-13.615946
16424,1981,1,1981-01-05,Albany,-16.166667,-25.000000,-7.111111,-15.790529,-24.121853,-7.052072
...,...,...,...,...,...,...,...,...,...,...
5569,2010,12,2010-12-27,Tulsa,-2.055556,-8.888889,7.777778,-2.034652,-8.789376,8.177132
5570,2010,12,2010-12-28,Tulsa,2.944444,-8.777778,8.277778,3.068649,-8.681155,8.717369
5571,2010,12,2010-12-29,Tulsa,7.611111,-1.000000,11.722222,7.997584,-0.970327,12.508714
5572,2010,12,2010-12-30,Tulsa,15.722222,4.388889,21.722222,17.086376,4.575143,24.391659


In [22]:
temp['mean_temp'].describe()

count    610812.000000
mean         15.068897
std           9.721414
min         -32.111111
25%           8.333333
50%          16.055556
75%          22.888889
max          43.222222
Name: mean_temp, dtype: float64

In [23]:
city_numbers = []
for year in year_temp:
    dict_year = {}
    df_year = temp[temp['year'] == year]
    numbers = len(df_year['city_name'].unique())
    dict_year[year] = numbers
    city_numbers.append(dict_year)

In [24]:
city_numbers

[{1981: 50},
 {1982: 50},
 {1983: 50},
 {1984: 50},
 {1985: 50},
 {1986: 50},
 {1987: 50},
 {1988: 50},
 {1989: 50},
 {1990: 50},
 {1991: 50},
 {1992: 50},
 {1993: 50},
 {1994: 50},
 {1995: 50},
 {1996: 50},
 {1997: 50},
 {1998: 50},
 {1999: 50},
 {2000: 50},
 {2001: 50},
 {2002: 50},
 {2003: 50},
 {2004: 50},
 {2005: 50},
 {2006: 50},
 {2007: 50},
 {2008: 50},
 {2009: 50},
 {2010: 50}]

In [25]:
import datetime

temp['date'] = pd.to_datetime(temp['date'])
temp.head()

/tmp/ipykernel_371365/108427414.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['date'] = pd.to_datetime(temp['date'])


,year,month,date,city_name,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
16420,1981,1,1981-01-01,Albany,-13.833333,-17.277778,-8.222222,-13.562737,-16.846720,-8.139343
16421,1981,1,1981-01-02,Albany,-6.277778,-17.222222,-2.777778,-6.233263,-16.793977,-2.759267
16422,1981,1,1981-01-03,Albany,-16.222222,-19.500000,-11.611111,-15.843406,-18.951129,-11.427102
16423,1981,1,1981-01-04,Albany,-20.722222,-25.000000,-13.888889,-20.104458,-24.121853,-13.615946
16424,1981,1,1981-01-05,Albany,-16.166667,-25.000000,-7.111111,-15.790529,-24.121853,-7.052072


In [26]:
temp_valid = temp[(temp['month'] == 7) | (temp['month'] == 8)]
temp_valid

,year,month,date,city_name,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
16601,1981,7,1981-07-01,Albany,21.611111,12.222222,30.000000,24.250801,13.069976,35.620631
16602,1981,7,1981-07-02,Albany,22.388889,12.777778,28.277778,25.241687,13.697092,33.152716
16603,1981,7,1981-07-03,Albany,20.666667,15.000000,28.277778,23.062659,16.244362,33.152716
16604,1981,7,1981-07-04,Albany,21.444444,15.000000,27.222222,24.039945,16.244362,31.677060
16605,1981,7,1981-07-05,Albany,19.611111,16.611111,24.388889,21.753492,18.132833,27.844160
...,...,...,...,...,...,...,...,...,...,...
5447,2010,8,2010-08-27,Tulsa,23.555556,12.222222,32.777778,26.749982,13.069976,39.771558
5448,2010,8,2010-08-28,Tulsa,26.000000,14.388889,34.388889,30.001517,15.537452,42.284057
5449,2010,8,2010-08-29,Tulsa,28.944444,17.222222,34.388889,34.098954,18.858992,42.284057
5450,2010,8,2010-08-30,Tulsa,28.500000,22.777778,35.000000,33.466881,25.741472,43.258653


In [27]:
temp_valid['mean_temp'].isna().sum()

0

In [28]:
temp_valid['max_temp'].isna().sum()

2

In [29]:
temp_valid['min_temp'].isna().sum()

14

In [30]:
min_AT_p85 = temp_valid.groupby(['city_name'])['AT_min'].apply(lambda x: np.percentile(x.dropna(), 85)).reset_index()
min_AT_p85.columns = ['city_name', 'AT_min_p85']
min_AT_p85

,city_name,AT_min_p85
0,Albany,26.388413
1,Albuquerque,22.233569
2,Allentown,22.993269
3,Atlanta,25.741472
4,Austin,28.655819
5,Baltimore,27.185933
6,Baton Rouge,27.844160
7,Birmingham,26.388413
8,Boston,23.480154
9,Buffalo,23.619764


In [31]:
climate

,year,month,date,city_name,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
17020,1961,1,1961-01-01,Albany,-3.277778,-7.000000,-0.388889,-3.259279,-6.943068,-0.351144
17021,1961,1,1961-01-02,Albany,-4.833333,-13.111111,-2.000000,-4.806708,-12.870244,-1.978794
17022,1961,1,1961-01-03,Albany,-7.777778,-12.611111,-2.611111,-7.705031,-12.389948,-2.592301
17023,1961,1,1961-01-04,Albany,-6.444444,-9.222222,-2.611111,-6.397263,-9.113764,-2.592301
17024,1961,1,1961-01-05,Albany,-7.500000,-15.888889,-2.611111,-7.433185,-15.526032,-2.592301
...,...,...,...,...,...,...,...,...,...,...
5449,2021,12,2021-12-27,Tulsa,15.777778,6.722222,22.777778,17.151448,7.044359,25.741472
5450,2021,12,2021-12-28,Tulsa,14.444444,10.000000,22.222222,15.601510,10.597245,25.028388
5451,2021,12,2021-12-29,Tulsa,7.666667,2.222222,22.222222,8.057404,2.321239,25.028388
5452,2021,12,2021-12-30,Tulsa,6.388889,2.222222,14.388889,6.688750,2.321239,15.537452


### Find the heatwave Count

In [32]:
def findList(city_name, AT_min_p85):
    temp_p85_list = []
    temp_p85_list.append(city_name)
    temp_p85_list.append(AT_min_p85)
    return temp_p85_list

In [35]:
min_AT_p85['temp_p85_list'] = min_AT_p85.apply(lambda x: findList(x['city_name'], x['AT_min_p85']), axis=1)
min_AT_p85.head()

,city_name,AT_min_p85,temp_p85_list
0,Albany,26.388413,"[Albany, 26.388413035890494]"
1,Albuquerque,22.233569,"[Albuquerque, 22.23356937081859]"
2,Allentown,22.993269,"[Allentown, 22.99326899576586]"
3,Atlanta,25.741472,"[Atlanta, 25.741471861415796]"
4,Austin,28.655819,"[Austin, 28.65581927853272]"


In [36]:
temp_p85_list = min_AT_p85['temp_p85_list'].tolist()
temp_p85_list[:5]

[['Albany', 26.388413035890494],
 ['Albuquerque', 22.23356937081859],
 ['Allentown', 22.99326899576586],
 ['Atlanta', 25.741471861415796],
 ['Austin', 28.65581927853272]]

In [37]:
def findHeatwaveCount(temp, city, temp_p85):
    df = temp[temp['city_name'] == city]
    c1 = df['AT_min'].ge(temp_p85)
    N = 2
    g = (c1 != c1.shift()).cumsum()
    df1 = df.assign(
        cnt=df.groupby(g).date.transform('count')
      , n=df.groupby(g).agg('cumcount')
      , g=g
    )
    c2 = df1.cnt.ge(N)
    c3 = df1.n.mod(N).eq(0) & df1.n.le(df1.cnt-N)
    df['flag'] = np.where(c1 & c2 & c3, 1, 0)
    del(df1)
    return df

In [38]:
climate

,year,month,date,city_name,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
17020,1961,1,1961-01-01,Albany,-3.277778,-7.000000,-0.388889,-3.259279,-6.943068,-0.351144
17021,1961,1,1961-01-02,Albany,-4.833333,-13.111111,-2.000000,-4.806708,-12.870244,-1.978794
17022,1961,1,1961-01-03,Albany,-7.777778,-12.611111,-2.611111,-7.705031,-12.389948,-2.592301
17023,1961,1,1961-01-04,Albany,-6.444444,-9.222222,-2.611111,-6.397263,-9.113764,-2.592301
17024,1961,1,1961-01-05,Albany,-7.500000,-15.888889,-2.611111,-7.433185,-15.526032,-2.592301
...,...,...,...,...,...,...,...,...,...,...
5449,2021,12,2021-12-27,Tulsa,15.777778,6.722222,22.777778,17.151448,7.044359,25.741472
5450,2021,12,2021-12-28,Tulsa,14.444444,10.000000,22.222222,15.601510,10.597245,25.028388
5451,2021,12,2021-12-29,Tulsa,7.666667,2.222222,22.222222,8.057404,2.321239,25.028388
5452,2021,12,2021-12-30,Tulsa,6.388889,2.222222,14.388889,6.688750,2.321239,15.537452


In [39]:
df = climate.merge(min_AT_p85, on='city_name', how='left')

In [40]:
df.head()

,year,month,date,city_name,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max,AT_min_p85,temp_p85_list
0,1961,1,1961-01-01,Albany,-3.277778,-7.000000,-0.388889,-3.259279,-6.943068,-0.351144,26.388413,"[Albany, 26.388413035890494]"
1,1961,1,1961-01-02,Albany,-4.833333,-13.111111,-2.000000,-4.806708,-12.870244,-1.978794,26.388413,"[Albany, 26.388413035890494]"
2,1961,1,1961-01-03,Albany,-7.777778,-12.611111,-2.611111,-7.705031,-12.389948,-2.592301,26.388413,"[Albany, 26.388413035890494]"
3,1961,1,1961-01-04,Albany,-6.444444,-9.222222,-2.611111,-6.397263,-9.113764,-2.592301,26.388413,"[Albany, 26.388413035890494]"
4,1961,1,1961-01-05,Albany,-7.500000,-15.888889,-2.611111,-7.433185,-15.526032,-2.592301,26.388413,"[Albany, 26.388413035890494]"


In [41]:
# Function to count heatwaves
def count_heatwaves(series, threshold):
    # Create a boolean series where True represents exceeding the threshold
    is_above_threshold = series > threshold

    # Identify groups of consecutive days above threshold
    # The diff and cumsum methods identify groups of consecutive True values
    heatwave_groups = (is_above_threshold.diff(1) != 0).cumsum()

    # Count the length of each group
    group_lengths = heatwave_groups[is_above_threshold].value_counts()

    # Count the groups that are at least two days long
    heatwave_count = (group_lengths >= 2).sum()

    return heatwave_count

In [42]:
# Apply the function to each year and fips
heatwave_counts = df.groupby(['year', 'month', 'city_name']).apply(lambda group: count_heatwaves(group['AT_min'], group['AT_min_p85'].iloc[0]))

# Reset the index
heatwave_counts = heatwave_counts.reset_index(name='heatwave_count')
heatwave_counts

,year,month,city_name,heatwave_count
0,1961,1,Albany,0
1,1961,1,Albuquerque,0
2,1961,1,Allentown,0
3,1961,1,Atlanta,0
4,1961,1,Austin,0
...,...,...,...,...
36088,2021,12,Seattle,0
36089,2021,12,St. Louis,0
36090,2021,12,Tampa,0
36091,2021,12,Tucson,0


In [46]:
heatwave_counts['heatwave_count'].describe()

count    36093.000000
mean         1.544482
std          1.544958
min          0.000000
25%          0.000000
50%          1.000000
75%          2.000000
max         11.000000
Name: heatwave_count, dtype: float64

In [43]:
heatwave_counts.to_csv('city_heatwave/min_apparent_temp/city_heatwave_count_monthly_1961_2021.csv')

In [44]:
heatwave_counts['heatwave_count'].unique()

array([0, 1, 4, 2, 3, 5, 8, 7, 6])

In [45]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame and has columns 'city', 'date', 'temperature'
# Assuming percentile_dict is your dictionary with 85th percentile temperatures for each city

# Sort by city and date
df = df.sort_values(['city_name', 'month', 'date'])

In [46]:
years = [year for year in range(1961, 2022)]

In [48]:
# Define a function to calculate heatwave durations for each city
def calculate_heatwave(df, years):
    pattern = []
    for year in years:
        # Create a boolean series where True indicates a day is part of a heatwave
        city_df = df[df['year'] == year] 
        for city in city_df['city_name'].unique():
            df_city = city_df[city_df['city_name'] == city]
            percentile = df_city['AT_min_p85']
            is_heatwave = df_city['AT_min'] > percentile

            # Identify where True values change to False and vice versa,
            # and assign each group of consecutive True values a unique identifier
            heatwave_id = (is_heatwave.diff() != 0).cumsum()

            # Group by the identifiers and count the size of each group
            heatwave_lengths = heatwave_id[is_heatwave].value_counts()

            # Filter out heatwaves that are less than 2 days long
            heatwave_lengths = heatwave_lengths[heatwave_lengths >= 2]

            # Create a DataFrame with the start date of each heatwave and its duration
            heatwaves = pd.DataFrame({
                'city_name': city,
                'duration': heatwave_lengths.values,
                'year': year
            })
            pattern.append(heatwaves)
    
    return pattern

In [50]:
pattern = calculate_heatwave(df, years)
heatwave_durations = pd.concat(pattern)
heatwave_durations

,city_name,duration,year
0,Albuquerque,2,1961
1,Albuquerque,2,1961
0,Allentown,3,1961
1,Allentown,2,1961
0,Austin,2,1961
...,...,...,...
5,Tampa,3,2021
6,Tampa,2,2021
7,Tampa,2,2021
0,Tucson,5,2021


In [51]:
# Function to calculate heatwave durations
def calculate_heatwave_durations(series, threshold):
    # Create a boolean series where True represents exceeding the threshold
    is_above_threshold = series > threshold

    # Identify groups of consecutive days above threshold
    # The diff and cumsum methods identify groups of consecutive True values
    heatwave_groups = (is_above_threshold.diff(1) != 0).cumsum()

    # Count the length of each group
    group_lengths = heatwave_groups[is_above_threshold].value_counts()

    # Filter for groups that are at least two days long, and return their lengths
    heatwave_durations = group_lengths[group_lengths >= 2]

    return heatwave_durations

In [52]:
# Apply the function to each year and fips
heatwave_durations = df.groupby(['year', 'city_name']).apply(lambda group: calculate_heatwave_durations(group['AT_min'], group['AT_min_p85'].iloc[0]))
# This will result in a series where each element is a series of heat wave durations
# To convert this into a DataFrame where each row represents a heat wave, you can do:
heatwave_durations = heatwave_durations.explode().reset_index(name='heatwave_duration')
heatwave_durations

,year,city_name,AT_min,heatwave_duration
0,1961,Albuquerque,4,2
1,1961,Albuquerque,12,2
2,1961,Allentown,8,3
3,1961,Allentown,4,2
4,1961,Austin,2,2
...,...,...,...,...
6347,2021,Tampa,20,3
6348,2021,Tampa,12,2
6349,2021,Tampa,32,2
6350,2021,Tucson,4,5


In [53]:
heatwave_durations.to_csv('city_heatwave/min_apparent_temp/city_heatwave_duration_1961_2021.csv')

In [54]:
df['date'] = pd.to_datetime(df['date'])

In [55]:
# Function to calculate heatwave dates
def calculate_heatwave_dates(series, threshold):
    # Create a boolean series where True represents exceeding the threshold
    is_above_threshold = series['AT_min'] > threshold

    # Identify groups of consecutive days above threshold
    # The diff and cumsum methods identify groups of consecutive True values
    heatwave_groups = (is_above_threshold.diff(1) != 0).cumsum()
    
    # Get the dates of each group
    group_dates = series.loc[is_above_threshold, 'date'].groupby(heatwave_groups)

    # Filter for groups that are at least two days long, and return their dates
    heatwave_dates = group_dates.filter(lambda dates: len(dates) >= 2)

    return heatwave_dates

In [56]:
# Apply the function to each year and fips
heatwave_dates = df.groupby(['year', 'city_name']).apply(lambda group: calculate_heatwave_dates(group[['date', 'AT_min']], group['AT_min_p85'].iloc[0]))
heatwave_dates

year  city_name           
1961  Albuquerque  44630     1961-06-29
                   44631     1961-06-30
                   44672     1961-08-10
                   44673     1961-08-11
      Allentown    66934     1961-07-23
                                ...    
2021  Tucson       1221241   2021-06-21
      Tulsa        1243570   2021-08-09
                   1243571   2021-08-10
                   1243572   2021-08-11
                   1243573   2021-08-12
Name: date, Length: 21385, dtype: datetime64[ns]

In [57]:
# The season length for each year and fips is the difference between the max and min date
season_lengths = heatwave_dates.groupby(level=['year','city_name']).apply(lambda dates: dates.max() - dates.min())
season_lengths = season_lengths.explode().reset_index(name='heatwave_season_length')
season_lengths

,year,city_name,heatwave_season_length
0,1961,Albuquerque,43 days
1,1961,Allentown,35 days
2,1961,Austin,1 days
3,1961,Birmingham,7 days
4,1961,Boston,55 days
...,...,...,...
2104,2021,Seattle,49 days
2105,2021,St. Louis,55 days
2106,2021,Tampa,137 days
2107,2021,Tucson,4 days


In [58]:
def findDaysNumber(days):
    days = str(days)
    return int(days.split()[0])+1

In [59]:
season_lengths['heatwave_length'] = season_lengths['heatwave_season_length'].apply(findDaysNumber)
season_lengths

,year,city_name,heatwave_season_length,heatwave_length
0,1961,Albuquerque,43 days,44
1,1961,Allentown,35 days,36
2,1961,Austin,1 days,2
3,1961,Birmingham,7 days,8
4,1961,Boston,55 days,56
...,...,...,...,...
2104,2021,Seattle,49 days,50
2105,2021,St. Louis,55 days,56
2106,2021,Tampa,137 days,138
2107,2021,Tucson,4 days,5


In [60]:
season_lengths = season_lengths.drop('heatwave_season_length', axis = 1)
season_lengths = season_lengths.rename(columns = {'heatwave_length': 'heatwave_season_length'})
season_lengths

,year,city_name,heatwave_season_length
0,1961,Albuquerque,44
1,1961,Allentown,36
2,1961,Austin,2
3,1961,Birmingham,8
4,1961,Boston,56
...,...,...,...
2104,2021,Seattle,50
2105,2021,St. Louis,56
2106,2021,Tampa,138
2107,2021,Tucson,5


In [61]:
season_lengths.to_csv('city_heatwave/min_apparent_temp/city_heatwave_season_length_1961_2021.csv')

In [62]:
def celsius_to_fahrenheit(c):
    f = c * 9/5 + 32
    return f

In [63]:
df['AT_min'] = df['AT_min'].apply(celsius_to_fahrenheit)
df['AT_min_p85'] = df['AT_min_p85'].apply(celsius_to_fahrenheit)

In [64]:
# Function to calculate average exceeded temperatures above threshold during heatwaves
def calculate_heatwave_exceeded_temp_avgs(series, threshold):
    # Create a boolean series where True represents exceeding the threshold
    is_above_threshold = series['AT_min'] > threshold

    # Identify groups of consecutive days above threshold
    # The diff and cumsum methods identify groups of consecutive True values
    heatwave_groups = (is_above_threshold.diff(1) != 0).cumsum()

    # Get the mean of exceeded temperatures (temperature - threshold) above threshold in each group
    group_exceeded_temp_avgs = (series.loc[is_above_threshold, 'AT_min'] - threshold).groupby(heatwave_groups).mean()

    # Filter for groups that are at least two days long, and return their exceeded temperature averages
    heatwave_exceeded_temp_avgs = group_exceeded_temp_avgs[group_exceeded_temp_avgs.index.map(is_above_threshold.groupby(heatwave_groups).size() >= 2)]

    return heatwave_exceeded_temp_avgs

In [65]:
# Apply the function to each year and fips
heatwave_exceeded_temp_avgs = df.groupby(['year', 'city_name']).apply(lambda group: calculate_heatwave_exceeded_temp_avgs(group[['AT_min']], group['AT_min_p85'].iloc[0]))
heatwave_exceeded_temp_avgs 

year  city_name    AT_min
1961  Albuquerque  4         1.367459
                   12        2.495151
      Allentown    4         4.946765
                   8         2.053965
      Austin       2         2.693875
                               ...   
2021  Tampa        18        2.422256
                   20        2.422256
                   32        2.422256
      Tucson       4         2.770709
      Tulsa        2         3.826874
Name: AT_min, Length: 6352, dtype: float64

In [66]:
# This will result in a series where each element is a series of heat wave exceeded temperature averages
# To convert this into a DataFrame where each row represents a heat wave, you can do:
heatwave_exceeded_temp_avgs = heatwave_exceeded_temp_avgs.explode().reset_index(name='heatwave_exceeded_temp_avg')
heatwave_exceeded_temp_avgs

,year,city_name,AT_min,heatwave_exceeded_temp_avg
0,1961,Albuquerque,4,1.367459
1,1961,Albuquerque,12,2.495151
2,1961,Allentown,4,4.946765
3,1961,Allentown,8,2.053965
4,1961,Austin,2,2.693875
...,...,...,...,...
6347,2021,Tampa,18,2.422256
6348,2021,Tampa,20,2.422256
6349,2021,Tampa,32,2.422256
6350,2021,Tucson,4,2.770709


In [67]:
heatwave_exceeded_temp_avgs = heatwave_exceeded_temp_avgs.drop('AT_min', axis = 1)
heatwave_exceeded_temp_avgs = heatwave_exceeded_temp_avgs.rename(columns = {'heatwave_exceeded_temp_avg': 'heatwave_intensity'})
heatwave_exceeded_temp_avgs

,year,city_name,heatwave_intensity
0,1961,Albuquerque,1.367459
1,1961,Albuquerque,2.495151
2,1961,Allentown,4.946765
3,1961,Allentown,2.053965
4,1961,Austin,2.693875
...,...,...,...
6347,2021,Tampa,2.422256
6348,2021,Tampa,2.422256
6349,2021,Tampa,2.422256
6350,2021,Tucson,2.770709


In [68]:
heatwave_exceeded_temp_avgs.to_csv('city_heatwave/min_apparent_temp/city_heatwave_intensity_1961_2021.csv')